In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from PIL import Image


In [ ]:
gpu = tf.config.experimental.list_physical_devices('GPU')
gpu

### Model for Classification

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('House_Room_Dataset', label_mode='categorical',image_size=(224, 224))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
data = data.map(lambda x,y: (x/255, y))
batch = data_iterator.next()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
from PIL import ImageFont
font = ImageFont.truetype("arial.ttf", 32)

model1 = Sequential()

model1.add(Conv2D(16,(3,3), 1, activation='relu', input_shape=(224,224,3)))
model1.add(MaxPooling2D())
model1.add(Dropout(0.5))

model1.add(Conv2D(32,(3,3), 1, activation='relu'))
model1.add(MaxPooling2D())
model1.add(Dropout(0.5))

model1.add(Conv2D(16,(3,3), 1, activation='relu'))
model1.add(MaxPooling2D())
model1.add(Dropout(0.5))

model1.add(Flatten())

model1.add(Dense(128,activation='relu'))
model1.add(Dense(5,activation='softmax'))

import visualkeras

visualkeras.layered_view(model1).show() # display using your system viewer
visualkeras.layered_view(model1, legend=True, font=font, scale_xy=1, scale_z=1, max_z=1000, to_file='output.png') # write to disk
visualkeras.layered_view(model1, legend=True, font=font, scale_xy=1, scale_z=1, max_z=1000, to_file='output.png').show()

In [ ]:
model1.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
hist1 = model1.fit(train, epochs= 30, validation_data=val, callbacks=[early_stopping])

In [ ]:
fig = plt.figure()
plt.plot(hist1.history['loss'], color='teal', label='Training loss')
plt.plot(hist1.history['val_loss'], color='orange', label='Validation loss')
fig.suptitle('Model Loss Values', fontsize=20)
plt.legend(loc="upper right")
plt.grid(axis='y')
plt.xlabel('Epoches')
plt.ylabel('Losses')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist1.history['accuracy'], color='teal', label='Train Accuracy')
plt.plot(hist1.history['val_accuracy'], color='orange', label='Validation Accuracy')
fig.suptitle('Model Accuracy Values', fontsize=20)
plt.legend(loc="upper left")
plt.grid(axis='y')
plt.xlabel('Epoch')
plt.ylabel('Losses')
plt.show()

In [ ]:
model1.save(os.path.join('models','Model-Classification.h5'))

In [ ]:
new_model = load_model(os.path.join('models', 'Model-Classification.h5'))
print(new_model)

conf_mat = tf.zeros((5, 5), dtype=tf.int32)

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = new_model.predict(X)
    y_pred = np.argmax(yhat, axis=1)
    y_true = np.argmax(y, axis=1)
    conf_mat += tf.math.confusion_matrix(y_true, y_pred, num_classes=5)

class_names = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']
conf_mat_norm = np.around(conf_mat.numpy() / conf_mat.numpy().sum(axis=1)[:, np.newaxis], decimals=2)

plt.figure(figsize=(8, 8))
sns.heatmap(conf_mat_norm, xticklabels=class_names, yticklabels=class_names, annot=True, fmt='.2f')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
test_loss1, test_acc1 = model1.evaluate(test)

In [ ]:
class_names = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']

img = cv2.imread('bed_test.jpg')
resize = tf.image.resize(img, (224, 224))
input_image = np.expand_dims(resize / 255, 0)
yhat = model1.predict(input_image)
predicted_class_index = np.argmax(yhat)
labels = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']
predicted_label = labels[predicted_class_index]

# Print the predicted label
print("Predicted label: ", predicted_label)

### VGG16 Model

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('House_Room_Dataset', label_mode='categorical',image_size=(224, 224))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
data = data.map(lambda x,y: (x/255, y))
batch = data_iterator.next()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
base_model_vgg16 = VGG16(input_shape=(224, 224, 3), include_top=False)

# Freeze the pre-trained layers
for layer in base_model_vgg16.layers:
    layer.trainable = False

# Add new classification layers on top of the pre-trained model
model2 = Sequential()
model2.add(base_model_vgg16)
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(5, activation='softmax'))

In [ ]:
model2.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
hist2 = model2.fit(train, epochs= 30, validation_data=val, callbacks=[early_stopping])

In [ ]:
fig = plt.figure()
plt.plot(hist2.history['loss'], color='teal', label='Training loss')
plt.plot(hist2.history['val_loss'], color='orange', label='Validation loss')
fig.suptitle('VGG16 Loss Values', fontsize=20)
plt.legend(loc="upper right")
plt.grid(axis='y')
plt.xlabel('Epoches')
plt.ylabel('Losses')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist2.history['accuracy'], color='teal', label='Train Accuracy')
plt.plot(hist2.history['val_accuracy'], color='orange', label='Validation Accuracy')
fig.suptitle('VGG16 Accuracy Values', fontsize=20)
plt.legend(loc="upper left")
plt.grid(axis='y')
plt.xlabel('Epoch')
plt.ylabel('Losses')
plt.show()

In [ ]:
model2.save(os.path.join('models','VGG16-Classification.h5'))

In [ ]:
test_loss2, test_acc2 = model2.evaluate(test)

In [ ]:
conf_mat = tf.zeros((5, 5), dtype=tf.int32)

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model2.predict(X)
    y_pred = np.argmax(yhat, axis=1)
    y_true = np.argmax(y, axis=1)
    conf_mat += tf.math.confusion_matrix(y_true, y_pred, num_classes=5)

class_names = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']
conf_mat_norm = np.around(conf_mat.numpy() / conf_mat.numpy().sum(axis=1)[:, np.newaxis], decimals=2)

plt.figure(figsize=(8, 8))
sns.heatmap(conf_mat_norm, xticklabels=class_names, yticklabels=class_names, annot=True, fmt='.2f')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
class_names = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']

img = cv2.imread('bath_test.jpg')
resize = tf.image.resize(img, (224, 224))
input_image = np.expand_dims(resize / 255, 0)
yhat = model2.predict(input_image)
predicted_class_index = np.argmax(yhat)
labels = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']
predicted_label = labels[predicted_class_index]

# Print the predicted label
print("Predicted label: ", predicted_label)

### Model VGG19

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('House_Room_Dataset', label_mode='categorical',image_size=(224, 224))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
data = data.map(lambda x,y: (x/255, y))
batch = data_iterator.next()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
base_model_vgg19 = VGG19(input_shape=(224, 224, 3), include_top=False)

# Freeze the pre-trained layers
for layer in base_model_vgg19.layers:
    layer.trainable = False

# Add new classification layers on top of the pre-trained model
model3 = Sequential()
model3.add(base_model_vgg19)
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(5, activation='softmax'))

In [ ]:
model3.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])
# Define the callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='min')

In [ ]:
hist3 = model3.fit(train, epochs= 30, validation_data=val, callbacks=[early_stopping])

In [ ]:
fig = plt.figure()
plt.plot(hist3.history['loss'], color='teal', label='Training loss')
plt.plot(hist3.history['val_loss'], color='orange', label='Validation loss')
fig.suptitle('VGG19 Loss Values', fontsize=20)
plt.legend(loc="upper right")
plt.grid(axis='y')
plt.xlabel('Epoches')
plt.ylabel('Losses')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist3.history['accuracy'], color='teal', label='Train Accuracy')
plt.plot(hist3.history['val_accuracy'], color='orange', label='Validation Accuracy')
fig.suptitle('VGG19 Accuracy Values', fontsize=20)
plt.legend(loc="upper left")
plt.grid(axis='y')
plt.xlabel('Epoch')
plt.ylabel('Losses')
plt.show()

In [ ]:
conf_mat = tf.zeros((5, 5), dtype=tf.int32)

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model3.predict(X)
    y_pred = np.argmax(yhat, axis=1)
    y_true = np.argmax(y, axis=1)
    conf_mat += tf.math.confusion_matrix(y_true, y_pred, num_classes=5)

class_names = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']
conf_mat_norm = np.around(conf_mat.numpy() / conf_mat.numpy().sum(axis=1)[:, np.newaxis], decimals=2)

plt.figure(figsize=(8, 8))
sns.heatmap(conf_mat_norm, xticklabels=class_names, yticklabels=class_names, annot=True, fmt='.2f')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
model3.save(os.path.join('models','VGG19-Classification.h5'))

In [ ]:
test_loss3, test_acc3 = model3.evaluate(test)

### Model Inception-V3

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('House_Room_Dataset', label_mode='categorical',image_size=(299, 299))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
data = data.map(lambda x,y: (x/255, y))
batch = data_iterator.next()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)+1
test_size = int(len(data)*.1)+1
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
base_model_inception = InceptionV3(input_shape=(299, 299, 3), include_top=False)

# Freeze the pre-trained layers
for layer in base_model_inception.layers:
    layer.trainable = False

# Add new classification layers on top of the pre-trained model
model4 = Sequential()
model4.add(base_model_inception)
model4.add(Flatten())
model4.add(Dense(128, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(5, activation='softmax'))

In [ ]:
model4.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])
# Define the callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='min')

In [ ]:
hist4 = model4.fit(train, epochs= 30, validation_data=val, callbacks=[early_stopping])

In [ ]:
fig = plt.figure()
plt.plot(hist4.history['loss'], color='teal', label='Training loss')
plt.plot(hist4.history['val_loss'], color='orange', label='Validation loss')
fig.suptitle('Inception-V3 Loss Values', fontsize=20)
plt.legend(loc="upper right")
plt.grid(axis='y')
plt.xlabel('Epoches')
plt.ylabel('Losses')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist4.history['accuracy'], color='teal', label='Train Accuracy')
plt.plot(hist4.history['val_accuracy'], color='orange', label='Validation Accuracy')
fig.suptitle('Inception-V3 Accuracy Values', fontsize=20)
plt.legend(loc="upper left")
plt.grid(axis='y')
plt.xlabel('Epoch')
plt.ylabel('Losses')
plt.show()

In [ ]:
conf_mat = tf.zeros((5, 5), dtype=tf.int32)

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model4.predict(X)
    y_pred = np.argmax(yhat, axis=1)
    y_true = np.argmax(y, axis=1)
    conf_mat += tf.math.confusion_matrix(y_true, y_pred, num_classes=5)

class_names = ['Bathroom', 'Bedroom', 'Dining', 'Kitchen', 'Living Room']
conf_mat_norm = np.around(conf_mat.numpy() / conf_mat.numpy().sum(axis=1)[:, np.newaxis], decimals=2)

plt.figure(figsize=(8, 8))
sns.heatmap(conf_mat_norm, xticklabels=class_names, yticklabels=class_names, annot=True, fmt='.2f')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
model4.save(os.path.join('models','InceptionV3-Classification.h5'))

In [ ]:
test_loss4, test_acc4= model4.evaluate(test)

### All models graphs

In [ ]:
# Get the last accuracy value for each model
acc1 = hist1.history['val_accuracy'][-1]
acc2 = hist2.history['val_accuracy'][-1]
acc3 = hist3.history['val_accuracy'][-1]
acc4 = hist4.history['val_accuracy'][-1]

# Create a list of labels for the x-axis ticks
labels = ['My Model', 'VGG-16', 'VGG-19', 'Inception-V3']

# Set the position of the x-axis ticks
x = [0, 1, 2, 3]

# Create the bar graph
fig, ax = plt.subplots()
rects = ax.bar(x, [acc1, acc2, acc3, acc4], align='center', alpha=0.5)

# Add labels, title, and grid
ax.set_ylabel('Accuracy')
ax.set_title('Models Validation Accuracy Values')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.grid(axis='y')

plt.show()

In [ ]:
# Create a list of labels for the x-axis ticks
labels = ['My Model', 'VGG-16', 'VGG-19', 'Inception-V3']

# Set the position of the x-axis ticks
x = [0, 1, 2, 3]

# Create the bar graph
fig, ax = plt.subplots()
rects = ax.bar(x, [test_acc1, test_acc2, test_acc3, test_acc4], align='center', alpha=0.5)

# Add labels, title, and grid
ax.set_ylabel('Test Accuracy')
ax.set_title('Models Test Accuracy Values')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.grid(axis='y')

plt.show()

In [ ]:
model1 = load_model('models/VGG16-Classification.h5')
model2 = load_model('models/VGG19-Classification.h5')
model3 = load_model('models/InceptionV3-Classification.h5')



In [ ]:
model1.evaluate(test)
model2.evaluate(test)

In [ ]:
model3.evaluate(test)